In [1]:
from transformers import BertForSequenceClassification, BertTokenizer
import torch
import torch.nn as nn
import pandas as pd
import re
from tqdm import tqdm

def clean_combine_news(unclean_news : pd.DataFrame) -> pd.DataFrame:
    temp = unclean_news["Top1"].copy()
    for i in range(2,26):
        temp += unclean_news[f"Top{i}"].fillna("")

    clean_news = []
    for i in range(0,temp.shape[0]):
        clean_news.append(re.sub("b[(')]",'',temp[i])) #remove b'
        clean_news[i]=re.sub('b[(")]','',clean_news[i]) #remove b"
        clean_news[i]=re.sub("\'",'',clean_news[i]) #remove \'

    return pd.DataFrame(clean_news, columns=["News"])

def clean_news(unclean : pd.DataFrame):
    unclean.fillna("", inplace=True)
    clean_news = []
    for i in range(0,unclean.shape[0]):
        clean_news.append(re.sub("b[(')]",'',unclean[i])) #remove b'
        clean_news[i]=re.sub('b[(")]','',clean_news[i]) #remove b"
        clean_news[i]=re.sub("\'",'',clean_news[i]) #remove \'
        clean_news[i]=re.sub("\"",'',clean_news[i]) #remove "

    return pd.DataFrame(clean_news, columns=["News"])

calculate = False

In [2]:
tokenizer = BertTokenizer.from_pretrained('ProsusAI/finbert')
model = BertForSequenceClassification.from_pretrained('ProsusAI/finbert')

In [3]:
stock_path = "E:/Users/chinm/Desktop/IIT Bhilai/z Completed Course/Sem5/Machine Learning/Project/Data/upload_DJIA_table.csv"
news_path = "E:/Users/chinm/Desktop/IIT Bhilai/z Completed Course/Sem5/Machine Learning/Project/Data/Combined_News_DJIA.csv"

stock_data = pd.read_csv(stock_path, index_col="Date").iloc[::-1]

news_data = pd.read_csv(news_path)
for i in range(1,26):
    news_data[f"Top{i}"] = clean_news(news_data[f"Top{i}"])
# news_data["News"] = clean_news(news_data)
# news_data.drop(columns=[f"Top{i}" for i in range(1,26)], inplace=True)
news_data.set_index("Date", inplace=True)

stock_data.head()

,Open,High,Low,Close,Volume,Adj Close
Date,,,,,,
2008-08-08,11432.089844,11759.959961,11388.040039,11734.320312,212830000,11734.320312
2008-08-11,11729.669922,11867.110352,11675.530273,11782.349609,183190000,11782.349609
2008-08-12,11781.700195,11782.349609,11601.519531,11642.469727,173590000,11642.469727
2008-08-13,11632.809570,11633.780273,11453.339844,11532.959961,182550000,11532.959961
2008-08-14,11532.070312,11718.280273,11450.889648,11615.929688,159790000,11615.929688


In [4]:
news_data.head(3)

,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,Top9,...,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
Date,,,,,,,,,,,,,,,,,,,,,
2008-08-08,0,Georgia downs two Russian warplanes as countri...,BREAKING: Musharraf to be impeached.,Russia Today: Columns of troops roll into Sout...,Russian tanks are moving towards the capital o...,"Afghan children raped with impunity, U.N. offi...",150 Russian tanks have entered South Ossetia w...,"Breaking: Georgia invades South Ossetia, Russi...",The enemy combatent trials are nothing but a s...,Georgian troops retreat from S. Osettain capit...,...,Georgia Invades South Ossetia - if Russia gets...,Al-Qaeda Faces Islamist Backlash,Condoleezza Rice: The US would not act to prev...,This is a busy day: The European Union has ap...,"Georgia will withdraw 1,000 soldiers from Iraq...",Why the Pentagon Thinks Attacking Iran is a Ba...,Caucasus in crisis: Georgia invades South Ossetia,Indian shoe manufactory - And again in a seri...,Visitors Suffering from Mental Illnesses Banne...,No Help for Mexicos Kidnapping Surge
2008-08-11,1,Why wont America and Nato help us? If they won...,Bush puts foot down on Georgian conflict,Jewish Georgian minister: Thanks to Israeli tr...,Georgian army flees in disarray as Russians ad...,Olympic opening ceremony fireworks faked,What were the Mossad with fraudulent New Zeala...,Russia angered by Israeli military sale to Geo...,An American citizen living in S.Ossetia blames...,Welcome To World War IV! Now In High Definition!,...,Israel and the US behind the Georgian aggression?,"Do not believe TV, neither Russian nor Georgia...",Riots are still going on in Montreal (Canada) ...,China to overtake US as largest manufacturer,War in South Ossetia [PICS],Israeli Physicians Group Condemns State Torture,Russia has just beaten the United States over...,Perhaps *the* question about the Georgia - Rus...,Russia is so much better at war,So this is what its come to: trading sex for f...
2008-08-12,0,Remember that adorable 9-year-old who sang at ...,Russia ends Georgia operation,If we had no sexual harassment we would have n...,Al-Qaeda is losing support in Iraq because of ...,Ceasefire in Georgia: Putin Outmaneuvers the West,Why Microsoft and Intel tried to kill the XO $...,Stratfor: The Russo-Georgian War and the Balan...,Im Trying to Get a Sense of This Whole Georgia...,The US military was surprised by the timing an...,...,U.S. troops still in Georgia (did you know the...,Why Russias response to Georgia was right,Gorbachev accuses U.S. of making a serious blu...,"Russia, Georgia, and NATO: Cold War Two",Remember that adorable 62-year-old who led you...,War in Georgia: The Israeli connection,All signs point to the US encouraging Georgia ...,Christopher King argues that the US and NATO a...,America: The New Mexico?,BBC NEWS | Asia-Pacific | Extinction by man no...


In [5]:
if calculate is True:
    pos, neg, neu = {}, {}, {}
    for i in tqdm(range(news_data.shape[0])):
        pos_temp, neg_temp, neu_temp = [], [], []
        for j in range(1,26):
            text = news_data.iloc[i][j]
            tokens = tokenizer.encode_plus(text)
            tokens = {'input_ids' : torch.Tensor([tokens['input_ids']]).long(),
                    'attention_mask' : torch.Tensor([tokens['attention_mask']]).int()}
            model_out = model(**tokens)
            out = model_out.logits.data
            out = torch.exp(out)/torch.sum(torch.exp(out))

            pos_temp.append(out[0][0])
            neg_temp.append(out[0][1])
            neu_temp.append(out[0][2])

        pos[i] = pos_temp
        neg[i] = neg_temp
        neu[i] = neu_temp
        if i%150 == 0:
            torch.save(pos, f"E:/Users/chinm/Desktop/IIT Bhilai/z Completed Course/Sem5/Machine Learning/Project/Transformers/Sentiment Cache/pos_{i}.pt")
            torch.save(neg, f"E:/Users/chinm/Desktop/IIT Bhilai/z Completed Course/Sem5/Machine Learning/Project/Transformers/Sentiment Cache/neg_{i}.pt")
            torch.save(neu, f"E:/Users/chinm/Desktop/IIT Bhilai/z Completed Course/Sem5/Machine Learning/Project/Transformers/Sentiment Cache/neu_{i}.pt")
        elif i == 1988:
            torch.save(pos, "E:/Users/chinm/Desktop/IIT Bhilai/z Completed Course/Sem5/Machine Learning/Project/Transformers/Sentiment Cache/pos.pt")
            torch.save(neg, "E:/Users/chinm/Desktop/IIT Bhilai/z Completed Course/Sem5/Machine Learning/Project/Transformers/Sentiment Cache/neg.pt")
            torch.save(neu, "E:/Users/chinm/Desktop/IIT Bhilai/z Completed Course/Sem5/Machine Learning/Project/Transformers/Sentiment Cache/neu.pt")
else:
    pos = torch.load("E:/Users/chinm/Desktop/IIT Bhilai/z Completed Course/Sem5/Machine Learning/Project/Transformers/Sentiment Cache/pos.pt")
    neg = torch.load("E:/Users/chinm/Desktop/IIT Bhilai/z Completed Course/Sem5/Machine Learning/Project/Transformers/Sentiment Cache/neg.pt")
    neu = torch.load("E:/Users/chinm/Desktop/IIT Bhilai/z Completed Course/Sem5/Machine Learning/Project/Transformers/Sentiment Cache/neu.pt")
    

In [6]:
cols = [f"Pos{i}" for i in range(1,26)]
pos_df = pd.DataFrame(pos).T
pos_df.columns = cols

cols = [f"Neg{i}" for i in range(1,26)]
neg_df = pd.DataFrame(neg).T
neg_df.columns = cols

cols = [f"Neu{i}" for i in range(1,26)]
neu_df = pd.DataFrame(neu).T
neu_df.columns = cols

sentiment = pd.concat([pos_df, neg_df, neu_df], axis = 1)
sentiment.index = news_data.index.copy()
del pos_df, neg_df, neu_df, cols
sentiment.head(2)

,Pos1,Pos2,Pos3,Pos4,Pos5,Pos6,Pos7,Pos8,Pos9,Pos10,...,Neu16,Neu17,Neu18,Neu19,Neu20,Neu21,Neu22,Neu23,Neu24,Neu25
Date,,,,,,,,,,,,,,,,,,,,,
2008-08-08,tensor(0.0665),tensor(0.0202),tensor(0.0439),tensor(0.0219),tensor(0.0151),tensor(0.0237),tensor(0.0511),tensor(0.0264),tensor(0.0124),tensor(0.0403),...,tensor(0.7933),tensor(0.4556),tensor(0.2799),tensor(0.0518),tensor(0.0707),tensor(0.8735),tensor(0.2044),tensor(0.5383),tensor(0.4871),tensor(0.4117)
2008-08-11,tensor(0.0717),tensor(0.0493),tensor(0.9326),tensor(0.0342),tensor(0.0259),tensor(0.0256),tensor(0.0188),tensor(0.0175),tensor(0.0557),tensor(0.0252),...,tensor(0.8067),tensor(0.8427),tensor(0.0458),tensor(0.0867),tensor(0.3033),tensor(0.5346),tensor(0.1832),tensor(0.8950),tensor(0.4938),tensor(0.9302)


In [7]:
sentiment.shape

(1989, 75)

In [8]:
cols = [f"Top{i}" for i in range(1,26)]
news_data_sentiment = pd.concat([news_data.drop(columns=cols), sentiment], axis=1).copy()
news_data_sentiment.head(2)

,Label,Pos1,Pos2,Pos3,Pos4,Pos5,Pos6,Pos7,Pos8,Pos9,...,Neu16,Neu17,Neu18,Neu19,Neu20,Neu21,Neu22,Neu23,Neu24,Neu25
Date,,,,,,,,,,,,,,,,,,,,,
2008-08-08,0,tensor(0.0665),tensor(0.0202),tensor(0.0439),tensor(0.0219),tensor(0.0151),tensor(0.0237),tensor(0.0511),tensor(0.0264),tensor(0.0124),...,tensor(0.7933),tensor(0.4556),tensor(0.2799),tensor(0.0518),tensor(0.0707),tensor(0.8735),tensor(0.2044),tensor(0.5383),tensor(0.4871),tensor(0.4117)
2008-08-11,1,tensor(0.0717),tensor(0.0493),tensor(0.9326),tensor(0.0342),tensor(0.0259),tensor(0.0256),tensor(0.0188),tensor(0.0175),tensor(0.0557),...,tensor(0.8067),tensor(0.8427),tensor(0.0458),tensor(0.0867),tensor(0.3033),tensor(0.5346),tensor(0.1832),tensor(0.8950),tensor(0.4938),tensor(0.9302)


In [9]:
data = pd.concat([news_data_sentiment, stock_data], axis=1)
data.head(3)

,Label,Pos1,Pos2,Pos3,Pos4,Pos5,Pos6,Pos7,Pos8,Pos9,...,Neu22,Neu23,Neu24,Neu25,Open,High,Low,Close,Volume,Adj Close
Date,,,,,,,,,,,,,,,,,,,,,
2008-08-08,0,tensor(0.0665),tensor(0.0202),tensor(0.0439),tensor(0.0219),tensor(0.0151),tensor(0.0237),tensor(0.0511),tensor(0.0264),tensor(0.0124),...,tensor(0.2044),tensor(0.5383),tensor(0.4871),tensor(0.4117),11432.089844,11759.959961,11388.040039,11734.320312,212830000,11734.320312
2008-08-11,1,tensor(0.0717),tensor(0.0493),tensor(0.9326),tensor(0.0342),tensor(0.0259),tensor(0.0256),tensor(0.0188),tensor(0.0175),tensor(0.0557),...,tensor(0.1832),tensor(0.8950),tensor(0.4938),tensor(0.9302),11729.669922,11867.110352,11675.530273,11782.349609,183190000,11782.349609
2008-08-12,0,tensor(0.0398),tensor(0.0293),tensor(0.0353),tensor(0.0081),tensor(0.1343),tensor(0.0282),tensor(0.0437),tensor(0.0402),tensor(0.0749),...,tensor(0.8498),tensor(0.1076),tensor(0.8954),tensor(0.9112),11781.700195,11782.349609,11601.519531,11642.469727,173590000,11642.469727


In [10]:
torch.save(data, "Data.pt")